In [1]:
import datatable as dt

In [2]:
biogrid_mv = dt.fread("data/BIOGRID-MV-Physical-3.5.187.tab3.txt",  header=True,)

In [3]:
with open(file="data/BIOGRID-MV-Physical-3.5.187.tab3.txt") as fi:
  ww = fi.readline().rstrip().split("\t")
biogrid_mv.names = ww



In [4]:
biogrid_mv_human = biogrid_mv[ list(map(lambda x,y: (x==9606)&(y==9606), biogrid_mv["Organism Interactor A"].to_list()[0], biogrid_mv["Organism Interactor B"].to_list()[0])),:]

In [61]:

edges = list(set(list(zip(map(str.upper, biogrid_mv_human["Official Symbol Interactor A"].to_list()[0]) , map(str.upper,biogrid_mv_human["Official Symbol Interactor B"].to_list()[0])))))

In [62]:
edges = list(set(list(map(lambda x: tuple(sorted(x)), edges))))

In [63]:
edges_zipped = list(zip(*edges))

In [64]:
all_nodes = sorted(set.union(set(edges_zipped[0]), set(edges_zipped[1])))

In [65]:
len(all_nodes)

10126

In [9]:
import torch
from torch_geometric.data import Data

In [66]:
node2ind = dict(zip(all_nodes, range(len(all_nodes))))

In [67]:
len(node2ind)

10126

In [68]:
edges_zipped_indices = [list(map(lambda x: node2ind[x], edges_zipped[0])),
                        list(map(lambda x: node2ind[x], edges_zipped[1]))]


In [69]:
edge_index = torch.tensor(edges_zipped_indices, dtype=torch.long)


In [37]:
import numpy as np
new_matrix = np.genfromtxt("gene_goslim_matrix.csv", delimiter = ",", max_rows = 5)

In [73]:
import pandas as pd
new_matrix = pd.read_csv("gene_goslim_matrix.csv")

In [74]:
new_matrix.set_index("Unnamed: 0", inplace=True)

In [83]:
ind2node = dict(zip(node2ind.values(), node2ind.keys()))

In [96]:
#new_matrix["MAP2K4"]
new_matrix[]
#list(map(lambda x: node2ind[x], new_matrix.columns.tolist()))

,MAP2K4,GATA2,RPA2,ARF1,ARF3,APP,APLP1,CITED2,EP300,APOB,...,IL1RL1,MASP1,TIMM22,NSDHL,SLC6A19,TMX2,RNR1,LOC102724652,DHCR24,SIGMAR1
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
GO:0005783_ENDOPLASMIC_RETICULUM,False,False,False,False,False,False,False,False,False,True,...,False,False,False,True,False,False,False,False,True,False
GO:0007610_BEHAVIOR,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
GO:0007165_SIGNAL_TRANSDUCTION,True,False,False,False,False,False,False,False,True,False,...,True,False,False,False,False,False,False,False,False,False
GO:0005198_STRUCTURAL_MOLECULE_ACTIVITY,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
GO:0007005_MITOCHONDRION_ORGANIZATION,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GO:0005576_EXTRACELLULAR_REGION,False,False,False,False,False,True,False,False,False,True,...,True,True,False,False,False,False,False,False,False,False
GO:0016032_VIRAL_REPRODUCTION,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
GO:0015031_PROTEIN_TRANSPORT,False,False,False,True,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [ ]:
from numpy import random
x = torch.tensor([random.randint(-1,2,size=len(all_nodes)),
     random.randint(-1,2,size=len(all_nodes)),
     random.randint(-1,2,size=len(all_nodes)),
     random.randint(-1,2,size=len(all_nodes))], dtype=torch.float).t()
y = torch.tensor(random.randint(0,4,len(all_nodes)), dtype=torch.long)
train_mask = torch.tensor(random.random_sample(size=len(all_nodes)) > .8, dtype=torch.bool)


In [ ]:
mydata = Data(edge_index=edge_index, x = x, y = y, train_mask = train_mask, num_classes = 4)

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(mydata.num_node_features, 2)
        self.conv2 = GCNConv(2, mydata.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        import pdb
        # pdb.set_trace()

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
mydata = mydata.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(mydata)
    loss = F.nll_loss(out[mydata.train_mask], mydata.y[mydata.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
a, pred = model(mydata).max(dim=1)

In [ ]:
data = mydata
correct = int(pred[torch.logical_not(data.train_mask)].eq(data.y[torch.logical_not(data.train_mask)]).sum().item())

In [ ]:
acc = correct / int(torch.logical_not(data.train_mask).sum())
acc

In [ ]:
from torch_geometric.datasets import Planetoid

planetset = Planetoid(root='/tmp/Cora', name='Cora')

planets = planetset[0]

In [ ]:
planets.edge_index.dtype